In [ ]:
%%capture
%pip install -U langchain
%pip install -U openai
%pip install -U ragas
%pip install -U arxiv
%pip install -U pymupdf
%pip install -U chromadb
%pip install -U tiktoken
%pip install -U accelerate
%pip install -U bitsandbytes
%pip install -U datasets
%pip install -U sentence_transformers
%pip install -U FlagEmbedding
%pip install -U ninja
%pip install -U flash_attn --no-build-isolation
%pip install -U tqdm
%pip install -U rank_bm25
%pip install -U transformers

In [ ]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

KeyboardInterrupt: Interrupted by user

In [ ]:
from langchain.document_loaders import ArxivLoader

base_docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()
len(base_docs)

In [ ]:
from pprint import pprint
for doc in base_docs:
  pprint(doc)
  break

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250)

docs = text_splitter.split_documents(base_docs)

vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [ ]:
print(len(docs))
print(docs[:2])

In [ ]:
print(max([len(chunk.page_content) for chunk in docs]))

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 2})

In [ ]:
relevant_docs = base_retriever.get_relevant_documents("What is Retrieval Augmented Generation?")
relevant_docs

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableParallel

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retriever =  RunnableParallel({"context":  base_retriever, "question":RunnablePassthrough()})

retrieval_augmented_qa_chain = retriever | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}

In [ ]:
question = "What is RAG?"

result = retrieval_augmented_qa_chain.invoke(question)

pprint(result)
